In [223]:
import os
import random
import warnings
from prophet.plot import add_changepoints_to_plot, plot_seasonality
import numpy as np
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import koreanize_matplotlib
# from sklearn.metrics import mean_absolute_percentage_error as mape
from tqdm.auto import tqdm
from sklearn.cluster import KMeans
# import logging
# logging.getLogger('Prophet').setLevel(logging.ERROR)
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', None)
warnings.filterwarnings(action='ignore')

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


seed_everything(42)  # Seed 고정

In [6]:
building_df = pd.read_csv('building_info.csv')
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [225]:
train_df.describe()

,b_num,temp,precip,w_s,hum,일조(hr),일사(MJ/m2),y,b_type,f_area,c_area,SUN_light,ESS_save,PCS,hour,sin_time,cos_time,day,weekend,month,THI,noon,CDH,km_cluster,km_0,km_1,km_2,km_3,km_4
count,204000.000000,204000.000000,204000.000000,204000.000000,204000.00000,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,2.040000e+05,2.040000e+05,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,204000.000000,204000.00000,204000.000000,204000.000000,204000.000000
mean,50.500000,25.543524,0.439489,2.134036,78.71640,0.210679,0.605373,2451.036462,5.190000,49.500000,47.530000,35.678400,86.561000,31.000000,11.500000,-1.842535e-17,-5.543278e-17,2.988235,0.282353,6.929412,75.419683,0.541667,-5.418965,2.030000,0.370000,0.04000,0.180000,0.010000,0.400000
std,28.866141,3.779523,2.539497,1.358110,15.08032,0.357338,0.920739,2440.648858,3.643345,28.866141,28.815849,89.696667,431.217345,146.079104,6.922204,7.071085e-01,7.071085e-01,1.991127,0.450145,0.793923,5.460606,0.498262,39.345829,1.768931,0.482805,0.19596,0.384188,0.099499,0.489899
min,1.000000,10.100000,0.000000,0.000000,13.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000e+00,-1.000000e+00,0.000000,0.000000,6.000000,50.481070,0.000000,-168.400000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,25.750000,23.400000,0.000000,1.200000,69.00000,0.000000,0.000000,1085.760000,2.000000,24.750000,22.750000,0.000000,0.000000,0.000000,5.750000,-7.071068e-01,-7.071068e-01,1.000000,0.000000,6.000000,72.551720,0.000000,-27.400000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,50.500000,25.700000,0.000000,2.000000,81.00000,0.000000,0.040000,1766.400000,5.000000,49.500000,47.500000,0.000000,0.000000,0.000000,11.500000,6.123234e-17,-6.123234e-17,3.000000,0.000000,7.000000,76.268500,1.000000,-1.200000,2.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,75.250000,28.100000,0.000000,2.800000,91.00000,0.300000,0.930000,2970.000000,8.000000,74.250000,72.250000,31.250000,0.000000,0.000000,17.250000,7.071068e-01,7.071068e-01,5.000000,1.000000,8.000000,79.366630,1.000000,22.200000,4.000000,1.000000,0.00000,0.000000,0.000000,1.000000
max,100.000000,37.100000,92.200000,13.300000,100.00000,1.000000,3.920000,25488.400000,11.000000,99.000000,97.000000,621.000000,3100.000000,1000.000000,23.000000,1.000000e+00,1.000000e+00,6.000000,1.000000,8.000000,87.789790,1.000000,100.300000,4.000000,1.000000,1.00000,1.000000,1.000000,1.000000


In [9]:
building_df['태양광용량(kW)'] = building_df['태양광용량(kW)'].replace('-', 0).astype(float)
building_df['ESS저장용량(kWh)'] = building_df['ESS저장용량(kWh)'].replace('-', 0).astype(float)
building_df['PCS용량(kW)'] = building_df['PCS용량(kW)'].replace('-', 0).astype(float)

# building_df['태양광용량(kW)'] = building_df['태양광용량(kW)'].astype(float)
# building_df['ESS저장용량(kWh)'] = building_df['ESS저장용량(kWh)'].astype(float)
# building_df['PCS용량(kW)'] = building_df['PCS용량(kW)'].astype(float)
# building_df

In [10]:
train_df = train_df.merge(building_df, on="건물번호", how="left")
test_df = test_df.merge(building_df, on="건물번호", how="left")
train_df

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,건물기타,110634.00,39570.00,0.0,0.0,0.0
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,건물기타,110634.00,39570.00,0.0,0.0,0.0
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88,건물기타,110634.00,39570.00,0.0,0.0,0.0
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76,건물기타,110634.00,39570.00,0.0,0.0,0.0
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40,건물기타,110634.00,39570.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04,호텔및리조트,57497.84,40035.23,0.0,0.0,0.0
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96,호텔및리조트,57497.84,40035.23,0.0,0.0,0.0
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12,호텔및리조트,57497.84,40035.23,0.0,0.0,0.0
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08,호텔및리조트,57497.84,40035.23,0.0,0.0,0.0


## 1. 가장 결과가 좋았던 외생변수 4종을 포함한 코드에 prameter 최적화를 수행한다
## 2. 랜포를 통한 파생변수 추출
## 3. 타 모델에서 결과가 나온 파생변수를 추가한다.

### 타 모델의 파생변수 추출

In [11]:
train_df.rename(columns={"num_date_time" : "num_date_time", "건물번호":"b_num", "일시" : "D&T", "기온(C)" : "temp", "강수량(mm)" : "precip", "풍속(m/s)" : "w_s" , "습도(%)" : "hum", "일조(hr)" : "일조(hr)", "일사(MJ/m2)" : "일사(MJ/m2)", "전력소비량(kWh)":"power", "건물유형":"b_type", "연면적(m2)":"f_area", "냉방면적(m2)":"c_area", "태양광용량(kW)": "SUN_light", "ESS저장용량(kWh)": "ESS_save", "PCS용량(kW)":"PCS" }, inplace=True)
test_df.rename(columns={"num_date_time" : "num_date_time", "건물번호":"b_num", "일시" : "D&T", "기온(C)" : "temp", "강수량(mm)" : "precip", "풍속(m/s)" : "w_s" , "습도(%)" : "hum", "일조(hr)" : "일조(hr)", "일사(MJ/m2)" : "일사(MJ/m2)", "전력소비량(kWh)":"power", "건물유형":"b_type", "연면적(m2)":"f_area", "냉방면적(m2)":"c_area", "태양광용량(kW)": "SUN_light", "ESS저장용량(kWh)": "ESS_save", "PCS용량(kW)":"PCS"} , inplace= True)
test_df

,num_date_time,b_num,D&T,temp,precip,w_s,hum,b_type,f_area,c_area,SUN_light,ESS_save,PCS
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72,건물기타,110634.00,39570.00,0.0,0.0,0.0
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72,건물기타,110634.00,39570.00,0.0,0.0,0.0
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75,건물기타,110634.00,39570.00,0.0,0.0,0.0
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78,건물기타,110634.00,39570.00,0.0,0.0,0.0
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77,건물기타,110634.00,39570.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100_20220831 19,100,20220831 19,22.5,0.0,0.9,84,호텔및리조트,57497.84,40035.23,0.0,0.0,0.0
16796,100_20220831 20,100,20220831 20,20.7,0.0,0.4,95,호텔및리조트,57497.84,40035.23,0.0,0.0,0.0
16797,100_20220831 21,100,20220831 21,20.2,0.0,0.4,98,호텔및리조트,57497.84,40035.23,0.0,0.0,0.0
16798,100_20220831 22,100,20220831 22,20.1,0.0,1.1,97,호텔및리조트,57497.84,40035.23,0.0,0.0,0.0


In [12]:
# 선형보간 실시
train_df["w_s"] = train_df["w_s"].interpolate()
train_df["hum"] = train_df["hum"].interpolate()

In [128]:
# 소수점 단위 절삭
train_df["c_area"] = train_df["c_area"].round(4)
train_df["f_area"] = train_df["f_area"].round(4)

In [136]:
lf = LabelEncoder()
train_df["c_area"] = lf.fit_transform(train_df["c_area"])
train_df["f_area"] = lf.fit_transform(train_df["f_area"])

,b_num,ds,temp,precip,w_s,hum,일조(hr),일사(MJ/m2),y,b_type,...,month,THI,noon,CDH,km_cluster,km_0,km_1,km_2,km_3,km_4
0,1,2022-06-01 00:00:00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,0,...,6,63.09388,0,-7.4,0,1,0,0,0,0
1,1,2022-06-01 01:00:00,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,0,...,6,62.46400,0,-15.4,0,1,0,0,0,0
2,1,2022-06-01 02:00:00,17.7,0.0,1.5,45.0,0.0,0.0,974.88,0,...,6,62.08735,0,-23.7,0,1,0,0,0,0
3,1,2022-06-01 03:00:00,16.7,0.0,1.4,48.0,0.0,0.0,953.76,0,...,6,60.89884,0,-33.0,0,1,0,0,0,0
4,1,2022-06-01 04:00:00,18.4,0.0,2.8,43.0,0.0,0.0,986.40,0,...,6,62.88788,0,-40.6,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,2022-08-24 19:00:00,23.1,0.0,0.9,86.0,0.5,0.0,881.04,11,...,8,72.38034,1,-19.6,4,0,0,0,0,1
203996,100,2022-08-24 20:00:00,22.4,0.0,1.3,86.0,0.0,0.0,798.96,11,...,8,71.21736,0,-20.2,4,0,0,0,0,1
203997,100,2022-08-24 21:00:00,21.3,0.0,1.0,92.0,0.0,0.0,825.12,11,...,8,69.79704,0,-22.3,4,0,0,0,0,1
203998,100,2022-08-24 22:00:00,21.0,0.0,0.3,94.0,0.0,0.0,640.08,11,...,8,69.41060,0,-25.1,4,0,0,0,0,1


In [218]:
train_df.columns

Index(['b_num', 'ds', 'temp', 'precip', 'w_s', 'hum', '일조(hr)', '일사(MJ/m2)',
       'y', 'b_type', 'f_area', 'c_area', 'SUN_light', 'ESS_save', 'PCS',
       'hour', 'sin_time', 'cos_time', 'day', 'weekend', 'month', 'THI',
       'noon', 'CDH', 'km_cluster', 'km_0', 'km_1', 'km_2', 'km_3', 'km_4'],
      dtype='object')

In [13]:
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)
def is_weekend(ds):
    date = pd.to_datetime(ds)
    if date.weekday() >= 5:
        return 1
    else:
        return 0
def is_noon(ds):
    date = pd.to_datetime(ds)
    if date.hour >= 7 and date.hour<20:
        return 1
    else:
        return 0
# 파생변수 생성 (train data)
train_df['D&T'] = pd.to_datetime(train_df['D&T'])
train_df['hour'] = train_df['D&T'].dt.hour
train_df['sin_time'] = np.sin(2*np.pi*train_df.hour/24)
train_df['cos_time'] = np.cos(2*np.pi*train_df.hour/24)
train_df['day'] = train_df['D&T'].dt.weekday
train_df['weekend'] = train_df['day'].isin([5,6]).astype(int)
train_df['month'] = train_df['D&T'].dt.month
train_df['THI'] = 9/5*train_df['temp'] - 0.55*(1-train_df['hum']/100)*(9/5*train_df['temp']-26)+32
train_df['weekend'] = train_df['D&T'].apply(is_weekend)
train_df['noon'] = train_df['D&T'].apply(is_noon)
cdhs = np.array([])
for num in range(1,101,1):
    temp = train_df[train_df['b_num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])

train_df['CDH'] = cdhs
train_df.drop("num_date_time", axis =1 , inplace=True)



In [96]:
# K-Means > train data
by_day = train_df.groupby(['b_num','day'])['power'].median().reset_index().pivot('b_num','day','power').reset_index()
by_hour = train_df.groupby(['b_num','hour'])['power'].median().reset_index().pivot('b_num','hour','power').reset_index().drop('b_num', axis = 1)
df = pd.concat([by_day, by_hour], axis= 1)
columns = ['b_num'] + ['day'+str(i) for i in range(7)] + ['hour'+str(i) for i in range(24)]
df.columns = columns
kmeans = KMeans(n_clusters=5, random_state = 42)
km_cluster = kmeans.fit_predict(df.iloc[:,1:])

df_clust = df.copy()
df_clust['km_cluster'] = km_cluster
# df_clust['km_cluster'] = df_clust['km_cluster'].map({})

train_df = train_df.merge(df_clust[['b_num','km_cluster']], on='b_num', how='left')
km_d = pd.get_dummies(train_df['km_cluster'], prefix='km', drop_first=False)

train_df = pd.concat([train_df, km_d], axis=1)
# prophet 형태로 전환
train_df.rename(columns={"D&T": "ds", "power": "y"}, inplace=True)
train_df = train_df.fillna(0)


KeyError: 'Column not found: power'

In [97]:
lf = LabelEncoder()
train_df["b_type"] = lf.fit_transform(train_df["b_type"])

In [15]:
train_df.head()

,b_num,ds,temp,precip,w_s,hum,일조(hr),일사(MJ/m2),y,b_type,...,month,THI,noon,CDH,km_cluster,km_0,km_1,km_2,km_3,km_4
0,1,2022-06-01 00:00:00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,건물기타,...,6,63.09388,0,-7.4,0,1,0,0,0,0
1,1,2022-06-01 01:00:00,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,건물기타,...,6,62.46400,0,-15.4,0,1,0,0,0,0
2,1,2022-06-01 02:00:00,17.7,0.0,1.5,45.0,0.0,0.0,974.88,건물기타,...,6,62.08735,0,-23.7,0,1,0,0,0,0
3,1,2022-06-01 03:00:00,16.7,0.0,1.4,48.0,0.0,0.0,953.76,건물기타,...,6,60.89884,0,-33.0,0,1,0,0,0,0
4,1,2022-06-01 04:00:00,18.4,0.0,2.8,43.0,0.0,0.0,986.40,건물기타,...,6,62.88788,0,-40.6,0,1,0,0,0,0


### RandomForest를 통한 Feature importance 추출

In [31]:
def get_score_splited_train(model, xtrain, xtest, ytrain, ytest):
    A = model.score(xtrain, ytrain)
    B = model.score(xtest,ytest)
    pred = model.predict(xtest)
    C = mape(ytest, pred)

    print(f"ACC train : {A:.4f}, test : {B:.4f}, mape : {C:.4f}" )

In [22]:
train_x = train_df.drop(columns=["y","ds"])
train_y = train_df['y']

In [29]:
xtrain, xtest, ytrain, ytest = train_test_split(train_x, train_y)

In [ ]:
best_rf = RandomForestRegressor(random_state=42, bootstrap=True, max_depth=None, min_samples_leaf= 1, min_samples_split=2, n_estimators=200)
best_rf.fit(xtrain,ytrain)

In [35]:
get_score_splited_train(best_rf, xtrain, xtest, ytrain, ytest)
prediction = best_rf.predict(xtest)
mape(ytest, prediction)
feature_importances = pd.DataFrame(best_rf.feature_importances_,
                                   index = xtrain.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

ACC train : 0.9992, test : 0.9944, mape : 72524537843998.7812
            importance
km_3          0.380319
km_1          0.311516
km_2          0.094076
noon          0.037508
c_area        0.022234
f_area        0.020095
b_num         0.015524
THI           0.011760
hour          0.011415
cos_time      0.011260
km_4          0.010700
day           0.010540
km_cluster    0.009116
SUN_light     0.008888
km_0          0.008292
sin_time      0.007775
weekend       0.007529
b_type        0.007191
CDH           0.004790
hum           0.001859
month         0.001687
temp          0.001544
w_s           0.001465
일사(MJ/m2)     0.001170
PCS           0.000698
ESS_save      0.000582
일조(hr)        0.000308
precip        0.000159


In [127]:
train_df["c_area"].round(4)

0         39570.00
1         39570.00
2         39570.00
3         39570.00
4         39570.00
            ...   
203995    40035.23
203996    40035.23
203997    40035.23
203998    40035.23
203999    40035.23
Name: c_area, Length: 204000, dtype: float64

In [231]:
'''{"num_date_time" : "num_date_time", "건물번호":"b_num", "일시" : "D&T", "기온(C)" : "temp", "강수량(mm)" : "precip", 
"풍속(m/s)" : "w_s" , "습도(%)" : "hum", "일조(hr)" : "일조(hr)", "일사(MJ/m2)" : "일사(MJ/m2)", 
"전력소비량(kWh)":"power", "건물유형":"b_type", "연면적(m2)":"f_area", "냉방면적(m2)":"c_area", "태양광용량(kW)": "SUN_light", 
"ESS저장용량(kWh)": "ESS_save", "PCS용량(kW)":"PCS" }'''

test_submission = pd.DataFrame(columns=['num_date_time', 'answer'])
# 나머지 0처리

holidays = pd.DataFrame(columns=['holiday', 'ds'])
# holidays["lower_window"] = 1
# holidays["upper_window"] = 1
mape_data = []

def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 100
for i in tqdm(range(1,101)):
    # 한건물당 168개씩 예측한다
    sample_train = train_df.loc[(train_df["b_num"] == i), ][:-168]
    sample_test = train_df.loc[(train_df["b_num"] == i), ][-168:].reset_index(drop=True)
    data = sample_train[[i for i in sample_train.columns if i != "일조(hr)" and i != "일사(MJ/m2)"]].reset_index(drop=True)
    test_data = sample_test[[i for i in sample_test.columns if i != "일조(hr)" and i != "일사(MJ/m2)"]].reset_index(drop=True)
    sample_x = test_data.drop("y" , axis=1)
    y = test_data["y"]

    m = Prophet(
        # growth: Any = 'linear',
        #      changepoints: Any = None,
        #      n_changepoints: Any = 25,
             changepoint_range = 1, # 5.716236755082271
        #      yearly_seasonality: Any = 'auto',
        #      weekly_seasonality: Any = 'auto',
        #      daily_seasonality = ,
             holidays = holidays,
             # seasonality_mode = 'multiplicative',
             # seasonality_prior_scale = 10,
             holidays_prior_scale =  0.007,# 5.798551926630998
             changepoint_prior_scale = 0.017, # 5.726094850564224
#              mcmc_samples = 84, # 나중에 다시 고려할 것
        #      interval_width =1,
        #      uncertainty_samples = 500,
        #      stan_backend: Any = None
    )
    '''
km_3          0.380319
km_1          0.311516
km_2          0.094076
noon          0.037508
c_area        0.022234
f_area        0.020095
b_num         0.015524
THI           0.011760
hour          0.011415
cos_time      0.011260
km_4          0.010700
day           0.010540
km_cluster    0.009116
SUN_light     0.008888
km_0          0.008292
sin_time      0.007775
weekend       0.007529
b_type        0.007191
CDH           0.004790
hum           0.001859
month         0.001687
temp          0.001544
w_s           0.001465
일사(MJ/m2)     0.001170
PCS           0.000698
ESS_save      0.000582
일조(hr)        0.000308
precip        0.000159
    '''
    # 계절성 추가
    m.add_seasonality(name='noon', period=1, fourier_order=84, condition_name='noon', prior_scale= 10)  # 3.68932757928076
    m.add_seasonality(name='weekend', period=7, fourier_order=84, condition_name='weekend', prior_scale= 7) # 3.6296482814114484
    
    # 공휴일 추가
    m.add_country_holidays(country_name='KR')
    # 외생변수 추가
    m.add_regressor('km_3')
    m.add_regressor('km_1') 
    m.add_regressor('km_2') # 6.357611052824596
#     m.add_regressor('c_area') # 옵티마이저 오류로 제외처리 -> label encoding 후 5.859738677367077
    m.add_regressor('f_area') # 옵티마이저 오류로 제외처리 -> label encoding 후 5.850376571825546
#     m.add_regressor('b_num') # 6.415231351641329
    m.add_regressor('THI') # 6.195088810269333
    m.add_regressor('hour') # 6.174984244049625
    m.add_regressor('cos_time') # 6.167108209487093
#     m.add_regressor('km_4') # 6.172539921486372
    m.add_regressor('day') # 6.100261553945959
#     m.add_regressor('km_cluster') # 6.104853937505138
#     m.add_regressor('SUN_light') # 6.108987779227074
#     m.add_regressor('km_0') # 6.108479479789926
#     m.add_regressor('sin_time') # 6.104529462878759
    m.add_regressor('b_type') # 5.918354697409236
#     m.add_regressor('CDH') # 5.971034075622254
    m.add_regressor('hum') # 5.90983672736866
    m.add_regressor('month') # 5.861836354704087
    m.add_regressor('temp') # 5.85962412110355
#     m.add_regressor('w_s') # 5.861515129127737
#     m.add_regressor('PCS') # 5.873309323319621
    m.add_regressor('ESS_save') # 5.8560206002232515
#     m.add_regressor('precip') # 5.8582046633179505

    m.fit(data
#           ,show_progress = False
          )
    
    # future에 데이터 삽입
    future = m.make_future_dataframe(periods=168, freq='1H')
    future['km_3'] = data['km_3']
    future['km_1'] = data['km_1']
    future['km_2'] = data['km_2']
    future['noon'] = data['noon'] 
#     future['c_area'] = data['c_area'] 
    future['f_area'] = data['f_area'] 
#     future['b_num'] = data['b_num'] 
    future['THI'] = data['THI'] 
    future['hour'] = data['hour']
    future['cos_time'] = data['cos_time']
#     future['km_4'] = data['km_4'] 
    future['day'] = data['day']
#     future['km_cluster'] = data['km_cluster']
#     future['SUN_light'] = data['SUN_light']
#     future['km_0'] = data['km_0']
#     future['sin_time'] = data['sin_time']
    future['weekend'] = data['weekend']
    future['b_type'] = data['b_type']
#     future['CDH'] = data['CDH']
    future['hum'] = data['hum']
    future['month'] = data['month']
    future['temp'] = data['temp']
#     future['w_s'] = data['w_s']
#     future['PCS'] = data['PCS']
    future['ESS_save'] = data['ESS_save']
#     future['precip'] = data['precip']
    
    # future 예측데이터 row에 데이터 삽입
    future.loc[future.shape[0] - 168:, 'km_3'] = sample_x['km_3'].values
    future.loc[future.shape[0] - 168:, 'km_1'] = sample_x['km_1'].values
    future.loc[future.shape[0] - 168:, 'km_2'] = sample_x['km_2'].values 
    future.loc[future.shape[0] - 168:, 'noon'] = sample_x['noon'].values    
#     future.loc[future.shape[0] - 168:, 'c_area'] = sample_x['c_area'].values 
    future.loc[future.shape[0] - 168:, 'f_area'] = sample_x['f_area'].values
#     future.loc[future.shape[0] - 168:, 'b_num'] = sample_x['b_num'].values 
    future.loc[future.shape[0] - 168:, 'THI'] = sample_x['THI'].values
    future.loc[future.shape[0] - 168:, 'hour'] = sample_x['hour'].values
    future.loc[future.shape[0] - 168:, 'cos_time'] = sample_x['cos_time'].values
#     future.loc[future.shape[0] - 168:, 'km_4'] = sample_x['km_4'].values
    future.loc[future.shape[0] - 168:, 'day'] = sample_x['day'].values
#     future.loc[future.shape[0] - 168:, 'km_cluster'] = sample_x['km_cluster'].values
#     future.loc[future.shape[0] - 168:, 'SUN_light'] = sample_x['SUN_light'].values
#     future.loc[future.shape[0] - 168:, 'km_0'] = sample_x['km_0'].values
#     future.loc[future.shape[0] - 168:, 'sin_time'] = sample_x['sin_time'].values
    future.loc[future.shape[0] - 168:, 'weekend'] = sample_x['weekend'].values
    future.loc[future.shape[0] - 168:, 'b_type'] = sample_x['b_type'].values
#     future.loc[future.shape[0] - 168:, 'CDH'] = sample_x['CDH'].values
    future.loc[future.shape[0] - 168:, 'hum'] = sample_x['hum'].values
    future.loc[future.shape[0] - 168:, 'month'] = sample_x['month'].values
    future.loc[future.shape[0] - 168:, 'temp'] = sample_x['temp'].values
#     future.loc[future.shape[0] - 168:, 'w_s'] = sample_x['w_s'].values
#     future.loc[future.shape[0] - 168:, 'PCS'] = sample_x['PCS'].values
    future.loc[future.shape[0] - 168:, 'ESS_save'] = sample_x['ESS_save'].values
#     future.loc[future.shape[0] - 168:, 'precip'] = sample_x['precip'].values    

    forecast = m.predict(future)

    answer = forecast[-168:].loc[:, ['ds', 'yhat_upper']].rename(
        columns={"yhat": "answer"}).reset_index(drop=True)

    answer["year"] = answer["ds"].dt.year.astype(str)
    answer["month"] = answer["ds"].dt.month.astype(str).str.zfill(2)
    answer["day"] = answer["ds"].dt.day.astype(str).str.zfill(2)
    answer["hour"] = answer["ds"].dt.hour.astype(str).str.zfill(2)
    answer["num_date_time"] = f"{i}_" + answer["year"] + answer["month"] + answer["day"] + ' ' + answer["hour"]
    test_submission = test_submission.append(answer)
    mape_data.append(SMAPE(y,forecast.loc[2040-168:,"yhat"].values))
print(f"SMAPE score is {sum(mape_data)/100}")
# test_submission = test_submission[["num_date_time", "answer"]]
# test_submission.set_index("num_date_time", inplace=True)

  0%|          | 0/100 [00:00<?, ?it/s]

09:11:21 - cmdstanpy - INFO - Chain [1] start processing
09:11:23 - cmdstanpy - INFO - Chain [1] done processing
09:11:24 - cmdstanpy - INFO - Chain [1] start processing
09:11:25 - cmdstanpy - INFO - Chain [1] done processing
09:11:26 - cmdstanpy - INFO - Chain [1] start processing
09:11:28 - cmdstanpy - INFO - Chain [1] done processing
09:11:29 - cmdstanpy - INFO - Chain [1] start processing
09:11:30 - cmdstanpy - INFO - Chain [1] done processing
09:11:32 - cmdstanpy - INFO - Chain [1] start processing
09:11:32 - cmdstanpy - INFO - Chain [1] done processing
09:11:33 - cmdstanpy - INFO - Chain [1] start processing
09:11:35 - cmdstanpy - INFO - Chain [1] done processing
09:11:36 - cmdstanpy - INFO - Chain [1] start processing
09:11:41 - cmdstanpy - INFO - Chain [1] done processing
09:11:42 - cmdstanpy - INFO - Chain [1] start processing
09:11:43 - cmdstanpy - INFO - Chain [1] done processing
09:11:44 - cmdstanpy - INFO - Chain [1] start processing
09:11:47 - cmdstanpy - INFO - Chain [1]

09:15:01 - cmdstanpy - INFO - Chain [1] done processing
09:15:02 - cmdstanpy - INFO - Chain [1] start processing
09:15:04 - cmdstanpy - INFO - Chain [1] done processing
09:15:05 - cmdstanpy - INFO - Chain [1] start processing
09:15:06 - cmdstanpy - INFO - Chain [1] done processing
09:15:07 - cmdstanpy - INFO - Chain [1] start processing
09:15:09 - cmdstanpy - INFO - Chain [1] done processing
09:15:10 - cmdstanpy - INFO - Chain [1] start processing
09:15:12 - cmdstanpy - INFO - Chain [1] done processing
09:15:13 - cmdstanpy - INFO - Chain [1] start processing
09:15:14 - cmdstanpy - INFO - Chain [1] done processing
09:15:15 - cmdstanpy - INFO - Chain [1] start processing
09:15:16 - cmdstanpy - INFO - Chain [1] done processing
09:15:17 - cmdstanpy - INFO - Chain [1] start processing
09:15:19 - cmdstanpy - INFO - Chain [1] done processing
09:15:20 - cmdstanpy - INFO - Chain [1] start processing
09:15:23 - cmdstanpy - INFO - Chain [1] done processing
09:15:24 - cmdstanpy - INFO - Chain [1] 

SMAPE score is 3.6917778139184736


In [230]:
print(f"The Last SMAPE score is {sum(mape_data)/100}")

The Last SMAPE score is 5.716236755082271


In [211]:
print(f"The Best SMAPE score is {sum(mape_data)/100}")

The Best SMAPE score is 3.6296482814114484


## 평가결과
1. linear 시작 = 6.425072103565819
2. weekend 추가 , fourier_order = 84 = 4.468917925091341
3. noon 추가, fourier_roder = 84 : 4.242046238975441
4. mcmc_sample = 30 : 4.275214333468416 <- 높이면 오르는데 시간이 너무 걸리므로 나중에 60이상으로 밤에 켜놓고 잘 것, 잘나오면 모델 저장도 프로펫 공홈보고 하기(show_progress = False 같이 켜야함)
5. holidays_prior_scale = 0.0173 : 4.064786963965926
6. changepoint_prior_scale = 0.017 : <b>3.690373125224021</b>
7. noon 아침 8시 - 오후 17시 :

In [226]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Alternative holiday of Chuseok,Alternative holiday of Chuseok_lower,Alternative holiday of Chuseok_upper,Alternative holiday of Hangeul Day,Alternative holiday of Hangeul Day_lower,Alternative holiday of Hangeul Day_upper,Birthday of the Buddha,Birthday of the Buddha_lower,Birthday of the Buddha_upper,Children's Day,Children's Day_lower,Children's Day_upper,Christmas Day,Christmas Day_lower,Christmas Day_upper,Chuseok,Chuseok_lower,Chuseok_upper,ESS_save,ESS_save_lower,ESS_save_upper,Hangeul Day,Hangeul Day_lower,Hangeul Day_upper,Independence Movement Day,Independence Movement Day_lower,Independence Movement Day_upper,Labour Day,Labour Day_lower,Labour Day_upper,Liberation Day,Liberation Day_lower,Liberation Day_upper,Lunar New Year's Day,Lunar New Year's Day_lower,Lunar New Year's Day_upper,Memorial Day,Memorial Day_lower,Memorial Day_upper,National Foundation Day,National Foundation Day_lower,National Foundation Day_upper,New Year's Day,New Year's Day_lower,New Year's Day_upper,THI,THI_lower,THI_upper,The day preceding of Chuseok,The day preceding of Chuseok_lower,The day preceding of Chuseok_upper,The day preceding of Lunar New Year's Day,The day preceding of Lunar New Year's Day_lower,The day preceding of Lunar New Year's Day_upper,The second day of Chuseok,The second day of Chuseok_lower,The second day of Chuseok_upper,The second day of Lunar New Year's Day,The second day of Lunar New Year's Day_lower,The second day of Lunar New Year's Day_upper,additive_terms,additive_terms_lower,additive_terms_upper,b_type,b_type_lower,b_type_upper,cos_time,cos_time_lower,cos_time_upper,daily,daily_lower,daily_upper,day,day_lower,day_upper,extra_regressors_additive,extra_regressors_additive_lower,extra_regressors_additive_upper,f_area,f_area_lower,f_area_upper,holidays,holidays_lower,holidays_upper,hour,hour_lower,hour_upper,hum,hum_lower,hum_upper,km_1,km_1_lower,km_1_upper,km_2,km_2_lower,km_2_upper,km_3,km_3_lower,km_3_upper,month,month_lower,month_upper,noon,noon_lower,noon_upper,temp,temp_lower,temp_upper,weekend,weekend_lower,weekend_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-06-01 00:00:00,723.511745,155.924046,333.738624,501.174052,969.005883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-33.996067,-57.595163,-10.142005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-476.146598,-729.163958,-256.579478,11.16891,-111.931749,129.348819,-1.313357,-20.426947,16.318505,-367.694232,-496.158181,-285.399746,-4.826303,-8.36148,-1.009784,-84.100044,-304.322435,125.346226,70.922882,-157.017316,274.38282,0.0,0.0,0.0,-7.369481,-18.005231,3.923820,-1.442150,-4.814613,2.227945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-22.591805,-30.016492,-15.118110,0.000000,0.000000,0.000000,-94.652674,-118.257296,-72.310353,0.0,0.0,0.0,-24.352322,-33.343069,-15.822705,0.0,0.0,0.0,247.365147
1,2022-06-01 01:00:00,723.498329,99.367846,280.808516,501.106169,969.017331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-36.503653,-61.843444,-10.890090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-535.681513,-797.203996,-314.266476,11.16891,-111.931749,129.348819,-1.268606,-19.730915,15.762465,-418.801502,-545.807727,-333.395115,-4.826303,-8.36148,-1.009784,-92.589236,-312.381494,117.679036,70.922882,-157.017316,274.38282,0.0,0.0,0.0,-6.728657,-16.439559,3.582618,-0.795560,-2.655976,1.229044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-22.591805,-30.016492,-15.118110,0.000000,0.000000,0.000000,-101.966445,-127.394985,-77.897743,0.0,0.0,0.0,-24.290775,-32.945956,-15.989366,0.0,0.0,0.0,187.816816
2,2022-06-01 02:00:00,723.484912,83.174129,267.919539,501.038285,969.028779,0.0,0.0,

In [39]:
future

106151     729.12
162449    1385.28
44512     1693.80
125298    2036.88
120128     846.36
           ...   
148325    3404.64
74532     5624.40
123247    3670.20
52536     3048.48
46286     1679.40
Name: y, Length: 153000, dtype: float64

In [43]:
m.train_holiday_names

0.050825041614038835

In [44]:
fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

            importance
km_3          0.387704
km_1          0.308216
km_2          0.093077
noon          0.037313
c_area        0.020971
f_area        0.020964
b_num         0.014945
hour          0.011953
km_4          0.011153
THI           0.010713
cos_time      0.010437
day           0.010016
km_cluster    0.009217
SUN_light     0.009123
weekend       0.007549
km_0          0.007547
sin_time      0.007327
b_type        0.007213
CDH           0.004986
month         0.001868
hum           0.001788
temp          0.001619
w_s           0.001482
일사(MJ/m2)     0.001198
PCS           0.000587
ESS_save      0.000557
일조(hr)        0.000301
precip        0.000174


In [ ]:
fig = m.plot_components(forecast)

,b_num,ds,temp,precip,w_s,hum,일조(hr),일사(MJ/m2),y,b_type,...,month,THI,noon,CDH,km_cluster,km_0,km_1,km_2,km_3,km_4
0,1,2022-06-01 00:00:00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,0,...,6,63.09388,0,-7.4,0,1,0,0,0,0
1,1,2022-06-01 01:00:00,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,0,...,6,62.46400,0,-15.4,0,1,0,0,0,0
2,1,2022-06-01 02:00:00,17.7,0.0,1.5,45.0,0.0,0.0,974.88,0,...,6,62.08735,0,-23.7,0,1,0,0,0,0
3,1,2022-06-01 03:00:00,16.7,0.0,1.4,48.0,0.0,0.0,953.76,0,...,6,60.89884,0,-33.0,0,1,0,0,0,0
4,1,2022-06-01 04:00:00,18.4,0.0,2.8,43.0,0.0,0.0,986.40,0,...,6,62.88788,0,-40.6,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,2022-08-24 19:00:00,23.1,0.0,0.9,86.0,0.5,0.0,881.04,11,...,8,72.38034,1,-19.6,4,0,0,0,0,1
203996,100,2022-08-24 20:00:00,22.4,0.0,1.3,86.0,0.0,0.0,798.96,11,...,8,71.21736,0,-20.2,4,0,0,0,0,1
203997,100,2022-08-24 21:00:00,21.3,0.0,1.0,92.0,0.0,0.0,825.12,11,...,8,69.79704,0,-22.3,4,0,0,0,0,1
203998,100,2022-08-24 22:00:00,21.0,0.0,0.3,94.0,0.0,0.0,640.08,11,...,8,69.41060,0,-25.1,4,0,0,0,0,1
